# Offer Generation Agent

In [7]:
from pathlib import Path

from langchain.agents import AgentExecutor, tool, create_tool_calling_agent
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain.pydantic_v1 import BaseModel, Field
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

from dotenv import load_dotenv

load_dotenv()

True

## Tools

### Calculate Promotions Tool

In [8]:
# class CalculatePromotionInput(BaseModel):
#     customer_insight_report: dict = Field(
#         description="A customer insight report that includes information on usage, charges, customer service calls, churn probability, and more.",
#         default=None,
#     )


# @tool("calculate_promotion_tool", args_schema=CalculatePromotionInput)
# def calculate_promotion_tool(customer_insight_report: dict) -> list[str]:

@tool("calculate_promotion_tool")
def calculate_promotion_tool() -> list[str]:
    """
    A tool to calculate available promotions for a customer using the
    customer insight report that is provided to you.
    """
    
    total_charge_usage = customer_insight_report["total_charge_usage"]
    total_minutes_usage = customer_insight_report["total_minutes_usage"]
    number_customer_service_calls = customer_insight_report["number_customer_service_calls"]
    sentiment_label = customer_insight_report["sentiment_label"]
    account_length_months = customer_insight_report["account_length_months"]
    churn_likelihood_percentage = customer_insight_report["churn_likelihood_percentage"]

    promotions = []

    if churn_likelihood_percentage > 0.6 or sentiment_label == "negative":
        promotions.append("Retention Offer: 20% discount for 6 months")
    if total_charge_usage in ["medium-high", "high"] or total_minutes_usage in ["medium-high", "high"]:
        promotions.append("Loyalty Offer: Free international minutes")
    if number_customer_service_calls >= 5:
        promotions.append("Service Credit: $10 off next bill")
    if sentiment_label == "positive" and churn_likelihood_percentage < 0.4:
        promotions.append("Upgrade Offer: Free device upgrade")
    if churn_likelihood_percentage <= 0.5 and total_charge_usage in ["medium", "medium-low"]:
        promotions.append("Bundle Offer: Add premium channels for $5/month")
    if account_length_months > 24:
        promotions.append("Anniversary Offer: 1 month free service")

    return promotions if promotions else ["General Offer: 10% discount for 3 months"]

In [41]:
customer_insight_report = {
    'account_length': '6 years, 7 months',
    'total_charge_usage': 'medium-low',
    'total_minutes_usage': 'medium',
    'number_customer_service_calls': 5,
    'sentiment_label': 'neutral',
    'account_length_months': 79,
    'churn_likelihood_percentage': 0.464,
    'support_ticket_summary': 'The customer is concerned about the escalating cost of voice services impacting turnover and is looking for alternative options, but hangs up before the TelCom agent can provide further assistance.'
}

In [10]:
# calculate_promotion_tool.invoke({"customer_insight_report" : customer_insight_report})
calculate_promotion_tool.invoke({})

['Service Credit: $10 off next bill',
 'Bundle Offer: Add premium channels for $5/month',
 'Anniversary Offer: 1 month free service']

In [96]:
customer_insight_report = {
    'account_length': '9 years, 9 months',
    'total_charge_usage': 'medium',
    'total_minutes_usage': 'medium',
    'number_customer_service_calls': 2,
    'sentiment_label': 'neutral',
    'account_length_months': 117,
    'churn_likelihood_percentage': 0.01,
    'support_ticket_summary': 'The customer has voice, text, and data services and is looking to upgrade their phone due to it being slow for a while.'
}

In [12]:
# calculate_promotion_tool.invoke({"customer_insight_report" : customer_insight_report})
calculate_promotion_tool.invoke({})

['Bundle Offer: Add premium channels for $5/month',
 'Anniversary Offer: 1 month free service']

In [98]:
customer_insight_report = {
    'account_length': '10 years, 5 months',
    'total_charge_usage': 'high',
    'total_minutes_usage': 'high',
    'number_customer_service_calls': 1,
    'sentiment_label': 'negative',
    'account_length_months': 125,
    'churn_likelihood_percentage': 0.792,
    'support_ticket_summary': 'Customer is frustrated with the poor customer service they have been receiving for their phone, despite not having any issues with the phone itself for the past three years.'
}

In [14]:
# calculate_promotion_tool.invoke({"customer_insight_report" : customer_insight_report})
calculate_promotion_tool.invoke({})

['Retention Offer: 20% discount for 6 months',
 'Loyalty Offer: Free international minutes',
 'Anniversary Offer: 1 month free service']

### Style Guidelines Retriever Tool

In [15]:
text_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=[
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ],
    strip_headers=False
)
texts = text_splitter.split_text(Path("data/style_guidelines_kb.md").read_text())

In [16]:
embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(texts, embeddings)
retriever = db.as_retriever(k=3)

In [17]:
style_guidelines_retriever_tool = create_retriever_tool(
    retriever,
    "style_guidelines_retriever_tool",
    "A tool for retrieving guidelines on creating personalized promotional email subjects and contents for customers, emphasizing a personable tone, customer focus, clarity, and a clear call-to-action, while maintaining brand alignment and accessibility. Make sure to search for relevent guidelines around structure, content, subject, etc. before writing the email.",
)

In [18]:
style_guidelines_retriever_tool.invoke("how to write a call to action?")

'### Email Structure  \n1. **Subject Line:**  \n- Capture attention.\n- Highlight the offer or benefit.\n- Keep it concise (50 characters or fewer).  \n**Examples:**  \n- "Stay with Us and Save 20%!"\n- "Thank You for 2 Years! Here’s a Special Gift."  \n2. **Greeting:**  \n- Use the customer’s name for personalization.  \n**Examples:**  \n- "Hi Alex,"\n- "Dear Maria,"  \n3. **Introduction:**  \n- Acknowledge the customer’s relationship with the company.\n- Set the stage for the offer.  \n**Examples:**  \n- "We appreciate you being a valued customer for the past 18 months."\n- "Thank you for your continued loyalty."  \n4. **Body:**  \n- Highlight the promotion and its benefits.\n- Tailor the message to the customer’s attributes (e.g., churn likelihood, usage).\n- Provide a reason for the offer (e.g., anniversary, feedback, loyalty).  \n**Examples:**  \n- "We noticed you’ve been making the most of your plan. As a thank-you, we’re offering you 20% off your bill for the next six months."\n

### Generate Email Tool

In [19]:
# class GenerateEmailInput(BaseModel):
#     customer_insight_report: dict = Field(
#         description="A customer insight report that includes information on usage, charges, customer service calls, churn probability, and more.",
#         default=None,
#     )
#     available_promotions: list = Field(
#         description="List of promotional offers available for this customer",
#         default=None,
#     )
#     style_guidelines: str = Field(
#         description="Relevant style guidelines for generating the personalized promotional email",
#         default=None,
#     )

class GenerateEmailInput(BaseModel):
    available_promotions: list = Field(
        description="List of promotional offers available for this customer",
        default=None,
    )
    style_guidelines: str = Field(
        description="Relevant style guidelines for generating the personalized promotional email subject",
        default=None,
    )


@tool("generate_email_tool", args_schema=GenerateEmailInput)
def generate_email_tool(
    # customer_insight_report: dict,
    available_promotions: list,
    style_guidelines: str
):
    """
    Tool to generate a personalized promotional email subject based on customer data, promotions, and style guidelines.
    Make sure to include the entirety of the relevant style guidelines.
    """
    
    llm = ChatOpenAI(name="gpt-4o-mini", temperature=0)
    prompt = PromptTemplate(
        template="""
        Generate a personalized promotional email subject and content based on the following customer data, available promotions, and style guidelines.
        Choose the single most relevant promotion based on the customer information.
        Be sure to adhere to the style guidelines.
        
        ## Customer data
        {customer_insight_report}
        
        ## Available promotions
        {available_promotions}
        
        ## Style guidelines
        {style_guidelines}
        """
    )
    chain = prompt | llm | StrOutputParser()
    return chain.invoke(
        {
            "customer_insight_report" : customer_insight_report,
            "available_promotions" : available_promotions,
            "style_guidelines" : style_guidelines
        }
    )

### Revise Email Tool

In [20]:
# class GenerateEmailInput(BaseModel):
#     customer_insight_report: dict = Field(
#         description="A customer insight report that includes information on usage, charges, customer service calls, churn probability, and more.",
#         default=None,
#     )
#     available_promotions: list = Field(
#         description="List of promotional offers available for this customer",
#         default=None,
#     )
#     style_guidelines: str = Field(
#         description="Relevant style guidelines for generating the personalized promotional email",
#         default=None,
#     )

class ReviseEmailInput(BaseModel):
    generated_email: str = Field(
        description="Generated promotional email",
        default=None,
    )
    simulated_customer_feedback: str = Field(
        description="Simulated customer feedback to improve the email effetiveness.",
        default=None,
    )
    available_promotions: list = Field(
        description="List of promotional offers available for this customer",
        default=None,
    )
    style_guidelines: str = Field(
        description="Relevant style guidelines for generating the personalized promotional email",
        default=None,
    )


@tool("revise_email_tool", args_schema=ReviseEmailInput)
def revise_email_tool(
    generated_email: str,
    simulated_customer_feedback: str,
    available_promotions: list,
    style_guidelines: str
):
    """
    Tool to revise personalized promotional email based on customer data, promotions, style guidelines
    and revised customer input. Update the email to better reflect the simulated customer
    feedback. Ensure that the changes do not change the factual accuracy
    of the available promotions or adherence to style guidelines.
    """
    
    llm = ChatOpenAI(name="gpt-4o-mini", temperature=0)
    prompt = PromptTemplate(
        template="""
        Revise the personalized promotional email based on customer data, promotions, style guidelines
        and revised customer input. Update the email to better reflect the simulated customer
        feedback. Ensure that the changes do not change the factual accuracy
        of the available promotions or adherence to style guidelines.
        
        ## Email Subject
        {email_subject}
        
        ## Customer data
        {customer_insight_report}
        
        ## Simulated customer feedback
        {simulated_customer_feedback}
        
        ## Available promotions
        {available_promotions}
        
        ## Style guidelines
        {style_guidelines}
        """
    )
    chain = prompt | llm | StrOutputParser()
    return chain.invoke(
        {
            "email_subject" : email_subject,
            "customer_insight_report" : customer_insight_report,
            "available_promotions" : available_promotions,
            "style_guidelines" : style_guidelines,
            "simulated_customer_feedback" : simulated_customer_feedback
        }
    )

### Customer Simulation Tool

In [121]:
class CustomerSimulationInput(BaseModel):
    generated_email: str = Field(
        description="Generated promotional email",
        default=None,
    )


@tool("customer_simulation_tool", args_schema=CustomerSimulationInput)
def customer_simulation_tool(generated_email: str):
    """
    Tool to see how well the customer will receive the email - use this to improve the relevance and tone.
    """
    
    llm = ChatOpenAI(name="gpt-4o-mini", temperature=0)
    prompt = PromptTemplate(
        template="""
        Act as the customer described in the below customer insight report. How successful
        would this email be? Describe the effectiveness and potential improvements in a few short sentences.
        
        ## Customer data
        {customer_insight_report}
        
        ## Generated email
        {generated_email}
        """
    )
    chain = prompt | llm | StrOutputParser()
    return chain.invoke(
        {
            "generated_email" : generated_email,
            "customer_insight_report" : customer_insight_report,
        }
    )

## Agent

In [122]:
from langchain.globals import set_debug

set_debug(False)

In [123]:
tools = [
    calculate_promotion_tool,
    style_guidelines_retriever_tool,
    generate_email_tool,
    revise_email_tool,
    customer_simulation_tool
]

In [124]:
llm = ChatOpenAI(name="gpt-4o-mini", temperature=0)

In [125]:
from langchain_core.tools.render import ToolsRenderer, render_text_description

In [126]:
OLD_PROMPT = """
Generate a personalized promotional email for the customer using the following flow:
1. Use the calculate_promotion_tool tool to get relevant offers.
2. Use the style_guidelines_retriever_tool for email structure, tone, and reference examples.
3. Use the generate_email_tool to generate the promotional email considering the style guidelines and customer information.
4. Use the customer_simulation_tool to ensure the email's relevance and get feedback.
5. Use the revise_email_tool to revise and update the email as needed.
6. Repeat the above steps as needed to create a high quality and relevant promotional email for the customer.
7. When you are satisfied with the email, output it without any pre or post amble.

Note: always use the customer_simulation_tool and revise_email_tool to make sure the email will be effective.
"""

In [127]:
prompt = ChatPromptTemplate.from_messages(
            [
                ("system", OLD_PROMPT),
                ("user", "{input}"),
                ("ai", "{agent_scratchpad}"),
            ]
        )

agent = AgentExecutor(
    agent=create_tool_calling_agent(
        llm=llm,
        tools=tools,
        prompt=prompt,
    ),
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

In [128]:
email = agent.invoke(
    input={"input" : OLD_PROMPT},
)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `calculate_promotion_tool` with `{}`


['Retention Offer: 20% discount for 6 months', 'Loyalty Offer: Free international minutes', 'Anniversary Offer: 1 month free service']
Invoking: `style_guidelines_retriever_tool` with `{'query': 'personalized promotional email structure and tone'}`


### Purpose  
This guide provides guidelines for crafting effective, personalized promotional email offers for customers. Emails should be engaging, clear, and reflect the specific promotional offers and customer attributes. Use this guide to ensure consistency in tone, language, and structure.  
---

### General Principles  
1. **Personable Tone:** Emails should feel conversational and friendly. Avoid overly formal language but maintain professionalism.
2. **Customer Focus:** Incorporate details specific to the customer, such as their usage patterns, account history, and sentiment profile.
3. **Clarity:** Ensure the email is easy to read. Avoid jargon and long, complex sentences.
4. **Call

In [129]:
print(email["output"])

Subject: Exclusive Offer for Our Valued Customer - 20% Discount for 6 Months!

Hi [Customer's Name],

We understand the frustration you've experienced with our customer service, and we want to make it right. Despite the challenges, we value your loyalty over the past 10 years and 5 months. As a token of our appreciation, we're offering you a 20% discount on your bill for the next six months.

This exclusive offer is our way of saying thank you for being a long-standing customer with high usage and minutes. We want to ensure that your experience with us is nothing short of exceptional.

Don't miss out on this opportunity to save on your monthly charges. Click below to claim your discount and continue enjoying our services hassle-free.

We look forward to continuing to serve you and improve your overall experience with IGZ Telecom.

Warm regards,
IGZ Telecom
